The goal:
We have two data sets, one on feature charachteristics of specific individuals and the other on match outcomes of games played between said indiviuals. The goal of this exercise is to (1) build a model that can predict the outcome of an unplayed game based solely on the features of two players, especially players that are not represented in the train-test data set and, using the feature weights learnt, (2) build a ranking system of strength and recommender model that will pair a new player with the most similarly strengthed player.

Assume that the only data available is the "player_data.csv" and "match_data_classification.csv"

First we will explore our player data to understand how many players's data we have access to, the features that are present, and what these look like.

In [60]:
#Importing the required libraries
import pandas as pd
import numpy as np
import os
#Defining the location of the CSV
DIR = '/Users/mohamed.alzarai/Desktop/Git/badminton_lads/Data Files (CSV)'
file_path = os.path.join(DIR,'player_data.csv')
#Loading data into the data frame
player_data = pd.read_csv(file_path)
#Understanding the first few data points
display(player_data.head())
#The first column is not adding anything to our analysis as it acts as a counter, hence we can drop it
player_data.drop(player_data.columns[0], axis=1, inplace=True)
#Understanding the first few data points
display(player_data.head())

,Unnamed: 0,Name,Height,Weight,Age,Experience,Dominant_hand,Gender,Historic Win Ratio,Reaction Time,Play Frequency,Athleticism,Serve Speed,Court Coverage,Vision
0,0,A0,180,57,18,9,left,male,0.139547,161.497106,0.740670,5,111.738777,18.438436,20/20
1,1,A1,145,63,25,17,left,male,0.716022,520.794984,0.419526,4,121.785314,26.188609,20/20
2,2,A2,188,60,31,11,right,male,0.215322,558.096477,0.102219,4,111.934328,26.949887,20/60
3,3,A3,142,79,35,4,right,male,0.552045,597.642799,0.618524,7,117.232701,13.845772,20/20
4,4,A4,142,92,25,10,left,male,0.866485,328.075735,0.453416,7,117.296644,17.403619,20/40


,Name,Height,Weight,Age,Experience,Dominant_hand,Gender,Historic Win Ratio,Reaction Time,Play Frequency,Athleticism,Serve Speed,Court Coverage,Vision
0,A0,180,57,18,9,left,male,0.139547,161.497106,0.740670,5,111.738777,18.438436,20/20
1,A1,145,63,25,17,left,male,0.716022,520.794984,0.419526,4,121.785314,26.188609,20/20
2,A2,188,60,31,11,right,male,0.215322,558.096477,0.102219,4,111.934328,26.949887,20/60
3,A3,142,79,35,4,right,male,0.552045,597.642799,0.618524,7,117.232701,13.845772,20/20
4,A4,142,92,25,10,left,male,0.866485,328.075735,0.453416,7,117.296644,17.403619,20/40


We see that there are features, in particular the Dominant_Hand, Gender and Vision that are not numerical hence can not be used for calculatuons. We will look to convert these to numerical viua one-hot encoding (OHE).

For the DominantHand, we will return a column such that left(hand)=0 and right(hand)=1
For the Gender, we will reutrn a column such that female=0, and male=1
For the vision, we will evaluate the fraction i.e. 20/20=1, 20/60=0.33, 20/40=0.5

In [ ]:
#Mapping values such that it is equiavalent to a OHE for dominant hand
player_data = pd.get_dummies(player_data, columns=['Dominant_hand'], drop_first=True)
#Mapping values such that it is equiavalent to a OHE for gender
player_data = pd.get_dummies(player_data, columns=['Gender'], drop_first=True)

#Converting both columns from booleans to binary for easier EDA and model building
player_data['Dominant_hand_right'] = player_data['Dominant_hand_right'].astype(int)
player_data['Gender_male'] = player_data['Gender_male'].astype(int)
display(player_data)



,Name,Height,Weight,Age,Experience,Historic Win Ratio,Reaction Time,Play Frequency,Athleticism,Serve Speed,Court Coverage,Vision,Dominant_hand_right,Gender_male
0,A0,180,57,18,9,0.139547,161.497106,0.740670,5,111.738777,18.438436,20/20,0,1
1,A1,145,63,25,17,0.716022,520.794984,0.419526,4,121.785314,26.188609,20/20,0,1
2,A2,188,60,31,11,0.215322,558.096477,0.102219,4,111.934328,26.949887,20/60,1,1
3,A3,142,79,35,4,0.552045,597.642799,0.618524,7,117.232701,13.845772,20/20,1,1
4,A4,142,92,25,10,0.866485,328.075735,0.453416,7,117.296644,17.403619,20/40,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,Z2,164,79,35,12,0.359651,629.687171,0.466009,7,129.626369,19.389808,20/60,1,0
178,Z3,153,73,28,14,0.741255,109.731941,0.553316,5,121.582859,18.351561,20/40,0,1
179,Z4,152,71,25,13,0.550647,666.078727,0.337571,7,126.431839,29.330165,20/80,0,0
180,Z5,169,60,29,6,0.184056,633.618718,0.990324,1,120.478888,26.781370,20/20,0,0


In [62]:
player_data['Vision']=pd.eval(player_data['Vision'])

ValueError: Length of values (101) does not match length of index (182)